In [1]:
import sys, os
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print("Added to sys.path:", repo_root)
from fixedincomelib import *
print("Fixed Income Library is loaded.")

Added to sys.path: c:\Users\Wanling Xie\FixedIncomeLib
Fixed Income Library is loaded.


### Build a Yield Curve

In [2]:
### de-serialize a pre-built model
path = 'serialized/yc_model.pickle'
yc_model : Model = qfReadModelFromFile(path)
# rebuild model for special dates
value_date = qfDisplayModelValueDate(yc_model)
data_collection = qfGetDataCollectionFromModel(yc_model)
build_method_collection = qfGetBuildMethodCollection(yc_model)
value_date_last_date = '2026-05-26'
yc_model_last_date = qfCreateModel(value_date_last_date, 'YIELD_CURVE', data_collection, build_method_collection)
value_date_expired = '2026-05-27'
yc_model_expired = qfCreateModel(value_date_expired, 'YIELD_CURVE', data_collection, build_method_collection)

### Build Product RFR Future

In [3]:
effecitve_date = '2026-05-26'
termination_date = '2026-08-26'
term = '3M'
future_convention = 'SOFR-FUTURE-3M'
long_or_short = 'LONG'
amount = 1
strike = 0.
prod_rfr_future = qfCreateProductRFRFuture(
    effecitve_date,
    termination_date,
    future_convention,
    long_or_short,
    amount,
    strike)

### Build Valuation Parameter Collection

In [4]:
### create funding index valuation parameters
vp_type = 'FUNDING INDEX PARAMETER'
content = {'Funding Index' : 'SOFR-1B'}
fi_vp = qfCreateValuationParameters(vp_type, content)
vp_collection = qfCreateValuationParametersCollection([fi_vp])

### Test Valuation

In [7]:
### test pv and cash
report = qfCreateValueReport(yc_model, prod_rfr_future, vp_collection, 'pvdetailed')
display(report.display())
report_last_date = qfCreateValueReport(yc_model_last_date, prod_rfr_future, vp_collection, 'pvdetailed')
display(report_last_date.display())
# # below SHOULD THROW AN ERROR MSG !!!
# report_expired = qfCreateValueReport(yc_model_expired, prod_rfr_future, vp_collection, 'pvdetailed')
# display(report_expired.display())

,Currency,Type,Value
0,USD,PV,2.386076e+07
1,USD,CASH,0.000000e+00


,Currency,Type,Value
0,USD,PV,0.000000e+00
1,USD,CASH,2.425747e+07


In [6]:
### test risk
rr_report = qfCreateValueReport(yc_model, prod_rfr_future, vp_collection, 'firstorderrisk')
display(rr_report)
rr_report_last_date = qfCreateValueReport(yc_model_last_date, prod_rfr_future, vp_collection, 'firstorderrisk')
display(rr_report_last_date)
# # below SHOULD THROW AN ERROR MSG !!!
# rr_report_expired = qfCreateValueReport(yc_model_expired, prod_rfr_future, vp_collection, 'firstorderrisk')
# display(rr_report_expired)

[array([-24544572.21266215,         0.        ,         0.        ,
                0.        ]),
 array([0., 0., 0.])]

[array([0., 0., 0., 0.]), array([0., 0., 0.])]

In [7]:
### test cf report
cf_report = qfCreateValueReport(yc_model, prod_rfr_future, vp_collection, 'cashflowsreport')
display(cf_report.display())
cf_report_last_date= qfCreateValueReport(yc_model_last_date, prod_rfr_future, vp_collection, 'cashflowsreport')
display(cf_report_last_date.display())
# # below SHOULD THROW AN ERROR MSG !!!
# cf_report_expired = qfCreateValueReport(yc_model_expired, prod_rfr_future, vp_collection, 'cashflowsreport')
# display(cf_report_expired.display())

,PRODUCT_TYPE,VALUATION_ENGINE_TYPE,LEG_ID,CASHFLOW_ID,PAY_OR_RECEIVE,NOTIONAL,PAY_DATE,FORECASTED_AMOUNT,PV,DISCOUNG FACTOR,FIXING_DATE,START_DATE,END_DATE,INDEX_OR_FIXED,INDEX_VALUE
0,PRODUCT_RFR_FUTURE,ValuationEngineProductRfrFuture,0,0,1.0,250000.0,"May 26th, 2026",2.417318e+07,2.386076e+07,0.987076,"August 26th, 2026","May 26th, 2026","August 26th, 2026",SOFRON Actual/360,0.033073


,PRODUCT_TYPE,VALUATION_ENGINE_TYPE,LEG_ID,CASHFLOW_ID,PAY_OR_RECEIVE,NOTIONAL,PAY_DATE,FORECASTED_AMOUNT,PV,DISCOUNG FACTOR,FIXING_DATE,START_DATE,END_DATE,INDEX_OR_FIXED,INDEX_VALUE
0,PRODUCT_RFR_FUTURE,ValuationEngineProductRfrFuture,0,0,1.0,250000.0,"May 26th, 2026",2.425747e+07,2.425747e+07,1.0,"August 26th, 2026","May 26th, 2026","August 26th, 2026",SOFRON Actual/360,0.029701


In [8]:
### par rate
simple_report = qfCreateValueReport(yc_model, prod_rfr_future, vp_collection, 'parrateorspread')
simple_report

0.03307287188337785